In [2]:
import ultralytics
from ultralytics import YOLO

In [3]:
from ultralytics.utils.downloads import download


In [4]:
import cv2 
import mediapipe as mp
import numpy as np
import time
import os

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [ ]:
# import cv2
# import mediapipe as mp
# import numpy as np
# import time

# # Initialize MediaPipe Pose
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose()

# # Threshold values
# STATE_THRESH = 90  # Knee angle threshold to determine squat state
# OFFSET_THRESH = 15  # Offset angle threshold for warning
# FEEDBACK_THRESH = 10  # Angle deviation threshold for feedback
# INACTIVE_THRESH = 3  # Seconds of inactivity before reset

# # Initialize state variables
# squat_counter_correct = 0
# squat_counter_incorrect = 0
# prev_state = None
# inactive_time = 0
# last_activity_time = time.time()

# # Function to calculate angle between three points
# def calculate_angle(a, b, c):
#     a = np.array(a)
#     b = np.array(b)
#     c = np.array(c)
    
#     ba = a - b
#     bc = c - b
    
#     cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
#     angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
#     return np.degrees(angle)

# # Function to check squat correctness
# def is_correct_squat(knee_angle, hip_angle, offset_angle):
#     if abs(offset_angle) > OFFSET_THRESH:
#         return False, "Incorrect Posture: Maintain Proper Alignment!"
    
#     if knee_angle < STATE_THRESH:
#         if hip_angle < 60:  # Example heuristic, adjust as needed
#             return False, "Incorrect Squat: Lower your hips properly!"
#         return True, "Good Squat!"
    
#     return None, ""  # No squat detected

# # Load video
# cap = cv2.VideoCapture('squat_video.mp4')

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # Convert frame to RGB
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
#     # Process the frame with MediaPipe Pose
#     result = pose.process(rgb_frame)

#     if result.pose_landmarks:
#         landmarks = result.pose_landmarks.landmark
#         h, w, _ = frame.shape

#         # Extract keypoints (left side used, you can extend for right)
#         shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w), 
#                     int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h))
#         hip = (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w), 
#                int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h))
#         knee = (int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w), 
#                 int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h))
#         ankle = (int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * w), 
#                  int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * h))

#         # Calculate angles
#         knee_angle = calculate_angle(hip, knee, ankle)
#         hip_angle = calculate_angle(shoulder, hip, knee)
#         offset_angle = calculate_angle((knee[0], knee[1]-50), hip, knee)  # Vertical offset

#         # Squat correctness check
#         squat_state, feedback = is_correct_squat(knee_angle, hip_angle, offset_angle)

#         # Count squats based on state changes
#         if squat_state is not None:
#             if squat_state and (prev_state is None or prev_state is False):
#                 squat_counter_correct += 1
#             elif not squat_state and (prev_state is None or prev_state is True):
#                 squat_counter_incorrect += 1
#             prev_state = squat_state
#             last_activity_time = time.time()  # Reset inactivity timer

#         # Display angles & squat count
#         cv2.putText(frame, f'Knee Angle: {int(knee_angle)}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
#         cv2.putText(frame, f'Hip Angle: {int(hip_angle)}', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
#         cv2.putText(frame, f'Squats Correct: {squat_counter_correct}', (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#         cv2.putText(frame, f'Squats Incorrect: {squat_counter_incorrect}', (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#         cv2.putText(frame, feedback, (50, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

#         # Draw landmarks & lines
#         cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
#         cv2.circle(frame, hip, 5, (0, 0, 255), -1)
#         cv2.circle(frame, knee, 5, (0, 255, 0), -1)
#         cv2.circle(frame, ankle, 5, (255, 0, 0), -1)
#         cv2.line(frame, shoulder, hip, (255, 255, 0), 2)
#         cv2.line(frame, hip, knee, (0, 255, 255), 2)
#         cv2.line(frame, knee, ankle, (255, 0, 255), 2)

#     # Compute inactivity time
#     inactive_time = time.time() - last_activity_time
#     if inactive_time > INACTIVE_THRESH:
#         squat_counter_correct, squat_counter_incorrect = 0, 0  # Reset counters
#         prev_state = None

#     # Show video
#     cv2.imshow('Squat Detection', frame)

#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# MediaPipe Setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Placeholder function to check if squat is correct using GNN
def check_correct_squat(joint_features):
    """
    Replace this function with a trained GNN model for classification.
    joint_features: List of angles and distances forming a feature vector.
    Returns: 'correct' or 'incorrect' squat.
    """
    return "correct"  # Placeholder output, replace with GNN inference

# OpenCV Video Capture
cap = cv2.VideoCapture("squat_video.mp4")  # Change to 0 for webcam

# Squat Counter Variables
counter = 0
stage = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(image)

    # Convert back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if result.pose_landmarks:
        # Extract key landmarks
        landmarks = result.pose_landmarks.landmark
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y]
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]

        # Calculate angles
        knee_angle = calculate_angle(hip, knee, ankle)
        hip_angle = calculate_angle(shoulder, hip, knee)

        # Display angles
        cv2.putText(image, f'Knee Angle: {int(knee_angle)}', (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(image, f'Hip Angle: {int(hip_angle)}', (50, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Squat detection logic
        if knee_angle < 90:  # Going down
            stage = "down"
        if knee_angle > 160 and stage == "down":  # Standing up
            stage = "up"
            counter += 1

        # GNN-based squat evaluation
        squat_status = check_correct_squat([hip_angle, knee_angle])

        # Display squat count and correctness
        cv2.putText(image, f'Count: {counter}', (50, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(image, f'Squat: {squat_status}', (50, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # Draw landmarks
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show output
    cv2.imshow("Squat Detection", image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# import cv2
# import mediapipe as mp
# import numpy as np
# import math

# # Initialize MediaPipe Pose model
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose()

# # Function to calculate angle between three points
# def calculate_angle(a, b, c):
#     """Calculate angle between three points (in degrees)."""
#     a = np.array(a)  
#     b = np.array(b)  
#     c = np.array(c)  
    
#     ba = a - b
#     bc = c - b
    
#     cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
#     angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
#     return np.degrees(angle)

# # Function to check if camera is sideways
# def is_camera_sideways(landmarks):
#     """Check if the shoulders are horizontally aligned (side view)."""
#     left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
#     right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

#     shoulder_distance_x = abs(left_shoulder.x - right_shoulder.x)
#     shoulder_distance_y = abs(left_shoulder.y - right_shoulder.y)

#     return shoulder_distance_x > shoulder_distance_y  # Side-view condition

# # Function to determine squat correctness & feedback
# def get_feedback(hip_angle, knee_angle, ankle_angle, correct_ranges):
#     """
#     Provide feedback based on the largest deviation from the correct squat angles.
#     """
#     ideal_hip_range = correct_ranges["hip"]
#     ideal_knee_range = correct_ranges["knee"]
#     ideal_ankle_range = correct_ranges["ankle"]

#     # Compute errors
#     hip_error = min(abs(hip_angle - ideal_hip_range[0]), abs(hip_angle - ideal_hip_range[1]))
#     knee_error = min(abs(knee_angle - ideal_knee_range[0]), abs(knee_angle - ideal_knee_range[1]))
#     # ankle_error = min(abs(ankle_angle - ideal_ankle_range[0]), abs(ankle_angle - ideal_ankle_range[1]))

#     # Find max error and give feedback
#     # max_error = max(hip_error, knee_error, ankle_error)
#     max_error = max(hip_error, knee_error)
#     if max_error == hip_error:
#         if hip_angle > ideal_hip_range[1]:
#             output.append(2)
#             return "Lower your hips" 
#         else:
#             output.append(3)
#             return "Raise your hips slightly"
        
#     elif max_error == knee_error:
#         if knee_angle > ideal_knee_range[1]:
#             output.append(4)
#             return "Bend your knees more"  
#         else:
#             output.append(5)
#             return"Straighten your knees slightly"
#     # elif max_error == ankle_error:
#     #     return "Adjust your ankle position"

#     return "Good squat!"
# def get_normalized_coords(landmark):
#                 return [landmark.x - mid_hip[0], landmark.y - mid_hip[1]]
# # Squat detection parameters
# correct_ranges = {
#     "hip": (60, 90),  # Ideal hip angle range
#     "knee": (70, 100),  # Ideal knee angle range
#     "ankle": (80, 110)  # Ideal ankle angle range
# }
# HIP_SQUAT_THRESHOLD = 160     # Hip angle threshold to detect squatting
# HIP_STANDING_THRESHOLD = 160  # Hip angle threshold to detect standing
# HIP_SQUAT_DETECTION_RANGE = (40, 160)  # Larger range for detecting squats

# # Video capture
# cap = cv2.VideoCapture("squat05.mp4")  # Change to 0 for webcam

# squat_count = 0
# squat_active = False
# output= []
# data_list = []
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert to RGB for MediaPipe
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image)

#     # Convert back to BGR for OpenCV
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     if results.pose_landmarks:
#         landmarks = results.pose_landmarks.landmark
#         mid_hip = np.array([(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x +
#                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x) / 2,
#                                 (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y +
#                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y) / 2])
#         left_hip = get_normalized_coords(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
#         right_hip = get_normalized_coords(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
#         left_knee = get_normalized_coords(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
#         right_knee = get_normalized_coords(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
#         left_ankle = get_normalized_coords(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
#         right_ankle = get_normalized_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

#         norm_shoulder = get_normalized_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])

#         if not is_camera_sideways(landmarks):
#             cv2.putText(image, "Face the camera sideways!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#             cv2.imshow("Squat Detection", image)
#             output.append(0)
#             continue  # Skip this frame if camera is not sideways
            
#         # Get required landmarks
#         hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
#                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
#         knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
#                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
#         ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
#                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
#         shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
#                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        
#         # Compute angles
#         hip_angle = calculate_angle(shoulder, hip, knee)
#         knee_angle = calculate_angle(hip, knee, ankle)
#         ankle_angle = calculate_angle(knee, ankle, [ankle[0], ankle[1] + 0.1])  # Vertical ref.

#         # Check for squat within defined hip angle range
#         if HIP_SQUAT_DETECTION_RANGE[0] <= hip_angle <= HIP_SQUAT_DETECTION_RANGE[1]:
#             if hip_angle < HIP_SQUAT_THRESHOLD and not squat_active:
#                 squat_active = True

#             if hip_angle > HIP_STANDING_THRESHOLD and squat_active:
#                 squat_active = False
#                 squat_count += 1  # Increment squat counter

#             # Get feedback **only if inside hip angle range**
#             feedback_text = get_feedback(hip_angle, knee_angle, ankle_angle, correct_ranges)
#         else:
#             feedback_text = "No squat detected"
#             output.append(1) 
#         # Display angles
#         cv2.putText(image, f"Hip Angle: {int(hip_angle)}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
#         cv2.putText(image, f"Knee Angle: {int(knee_angle)}", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
#         cv2.putText(image, f"Ankle Angle: {int(ankle_angle)}", (50, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
#         cv2.putText(image, f"Squat Count: {squat_count}", (50, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

#         # Display feedback
#         cv2.putText(image, feedback_text, (50, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
#         data_list.append([
#                 left_hip[0], left_hip[1], right_hip[0], right_hip[1],
#                 left_knee[0], left_knee[1], right_knee[0], right_knee[1],
#                 left_ankle[0], left_ankle[1], right_ankle[0], right_ankle[1],output[-1]
#             ]) 
        
#     # Show frame
#     cv2.imshow("Squat Detection", image)

#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [8]:
import pandas as pd
df = pd.DataFrame(data_list, columns=[
    "left_hip_x", "left_hip_y", "right_hip_x", "right_hip_y",
    "left_knee_x", "left_knee_y", "right_knee_x", "right_knee_y",
    "left_ankle_x", "left_ankle_y", "right_ankle_x", "right_ankle_y",
    "label"
])
df.to_csv("squat_data.csv", index=False)

print("Dataset saved to 'squat_data.csv'")

Dataset saved to 'squat_data.csv'


In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader  
from torch_geometric.nn import GCNConv
from torch_geometric.data import  Dataset
import pandas as pd

class SquatDataset(Dataset):
    def __init__(self, csv_file, transform=None, pre_transform=None):
        super().__init__(None, transform, pre_transform)
        self.df = pd.read_csv(csv_file)
        
    def len(self):
        return len(self.df)

    def get(self, idx):
        row = self.df.iloc[idx]

        # Extract node features (6 points, each with x, y)
        x = torch.tensor(row[:-1].values.reshape(6, 2), dtype=torch.float)

        # Define edges (fully connected graph for now)
        edges = [
            (0, 1), (0, 2), (2,4), (1, 3),(3,5)
        ]
        edges += [(j, i) for i, j in edges]

        # Convert to tensor
        edge_index = torch.tensor(edges, dtype=torch.long).t()

        # Label
        y = torch.tensor([row.iloc[-1]], dtype=torch.long)  # Use .iloc to get last item by position

        return Data(x=x, edge_index=edge_index, y=y)
        

In [10]:

dataset = SquatDataset("squat_dataset.csv")
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

# Define dataset split (e.g., 80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Define DataLoaders for PyTorch Geometric
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)



In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Define the GNN model

class SquatGNN(torch.nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32, output_dim=6):
        super(SquatGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, 2*hidden_dim)
        self.conv2 = GCNConv(2*hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)  
        self.dropout= nn.Dropout(0.3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # x = torch.reshape(x,(-1,6,2)).to(device)
        # edge_index = torch.reshape(edge_index,(2,10,-1)).to(device)
        
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.conv2(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.conv3(x, edge_index)
        x = F.elu(x)
        x = self.dropout(x)
        x = global_mean_pool(x, data.batch)  # Global mean pooling
        x = self.fc(x)
        
        return x
        return (F.log_softmax(x))  # Use log-softmax for NLLLoss
        return [x,edge_index]

Using device: cuda


In [12]:
model = SquatGNN().to(device)

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
# loss_fn = nn.NLLLoss()
loss_fn = nn.CrossEntropyLoss()

# Training loop
epochs = 40
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data) 
        #break
        loss = loss_fn(out, data.y)  
        #break
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

print("Training complete!")


Epoch 1, Loss: 98.2563
Epoch 2, Loss: 36.3274
Epoch 3, Loss: 30.8957
Epoch 4, Loss: 25.3495
Epoch 5, Loss: 25.5369
Epoch 6, Loss: 19.7651
Epoch 7, Loss: 25.9138
Epoch 8, Loss: 21.4535
Epoch 9, Loss: 29.7243
Epoch 10, Loss: 19.3776
Epoch 11, Loss: 19.7951
Epoch 12, Loss: 23.4760
Epoch 13, Loss: 23.4900
Epoch 14, Loss: 17.8586
Epoch 15, Loss: 19.3943
Epoch 16, Loss: 23.1708
Epoch 17, Loss: 16.8538
Epoch 18, Loss: 17.5717
Epoch 19, Loss: 18.7093
Epoch 20, Loss: 18.1225
Epoch 21, Loss: 21.3452
Epoch 22, Loss: 19.7122
Epoch 23, Loss: 18.5236
Epoch 24, Loss: 24.6279
Epoch 25, Loss: 17.1394
Epoch 26, Loss: 22.1856
Epoch 27, Loss: 17.9609
Epoch 28, Loss: 22.2531
Epoch 29, Loss: 17.3014
Epoch 30, Loss: 17.2984
Epoch 31, Loss: 16.2882
Epoch 32, Loss: 19.5512
Epoch 33, Loss: 17.7541
Epoch 34, Loss: 15.5685
Epoch 35, Loss: 21.4511
Epoch 36, Loss: 17.1107
Epoch 37, Loss: 14.6793
Epoch 38, Loss: 18.1255
Epoch 39, Loss: 19.0194
Epoch 40, Loss: 17.1817
Training complete!


In [14]:
import torch

def evaluate(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No gradient updates during validation
        for data in val_loader:
            data = data.to(device)
            output = model(data)  # Forward pass
            preds = output.argmax(dim=1)  # Get predicted class
            correct += (preds == data.y).sum().item()
            total += data.y.size(0)

    accuracy = 100 * correct / total
    return accuracy
print(evaluate(model,val_loader,device))

95.14563106796116


In [15]:
import pandas as pd
df = pd.read_csv("squat_dataset.csv")

In [16]:
df[df.columns[:-1]].iloc[0].values

array([  -0.022024,  -0.0066378,    0.022024,   0.0066378,   -0.034584,     0.15786,    0.036604,     0.16804,   -0.081443,     0.31659,     0.03867,     0.34752])